### Restart and run all cells

#### csv files: sell-alert

In [1]:
import calendar
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine
from math import ceil, floor, trunc

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
con_lite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
today

datetime.date(2022, 1, 17)

In [2]:
def number_of_spread(x):
    maxp = x[1]
    minp = x[10]
    div = 0.0
    gap = 0.0
    spd = 0
    spd1 = 0
    spd2 = 0
    if (maxp > minp): 
        mult = 1
    else:
    	tmpp = minp
    	minp = maxp
    	maxp = tmpp
    	mult = -1
        
    gap = round(maxp - minp,2)   
    
    if (maxp > 400.00):
        div = 2.0
        boundary = div * 200
        if ((minp < boundary)):
            spd1 = trunc((boundary - minp) / 1.00)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 200.00):
        div = 1.0
        boundary = div * 200
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.50)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 100.00):
        div = 0.50
        boundary = div * 200
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.25)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 25.00):
        div = 0.25
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.10)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div)
    elif (maxp > 10.00):
        div = 0.10
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.05)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            div = 0.10            
            spd = trunc(gap / div)
    elif (maxp > 5.00):
        div = 0.05
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.02)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2           
        else:
            div = 0.05            
            spd = ceil(gap / div)
    elif (maxp > 2.00):
        div = 0.02
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.01)
            spd2 = trunc((maxp - boundary) / div)
            spd = spd1 + spd2
        else:          
            spd = (gap / div)
    else:
        div = 0.01
        boundary = div * 100        
        spd = trunc(gap / div) 
        
    return spd * mult

In [3]:
format_dict = {
               'fm_price':'{:.2f}','to_price':'{:.2f}',    
               'max_price':'{:.2f}','min_price':'{:.2f}',
               'target':'{:.2f}','max':'{:.2f}','max52':'{:.2f}','css_price':'{:.2f}','amount':'{:,.2f}',  
               'unit_cost':'{:.2f}','cost_amt':'{:,.2f}',
               'dividend':'{:.2f}','dividend_amount':'{:,.2f}',    
               'cost_amt':'{:,.2f}','div_amt':'{:,.2f}','cost_pct':'{:.2f}', 
               'buy_target':'{:.2f}','sell_target':'{:.2f}',
               'volume':'{:,.2f}','beta':'{:.2f}','cost':'{:,.2f}',
               'qty':'{:,}','available_qty':'{:,}','shares':'{:,}'}
pd.set_option("display.max_rows", None)

In [4]:
sql = '''
SELECT T.name,sell_target AS target, sh, T.qty,T.available_qty,cost,
A.fm_date, A.to_date, days, fm_price, to_price, ttl_spread AS ttl, 
A.max_price AS max, T.max_price AS max52, consensus_price AS css_price,
sell_target * T.qty AS amount, T.status
FROM stocks T
JOIN sales A
ON T.name = A.name
LEFT JOIN consensus C
ON T.name = C.name
WHERE T.status IN ('I','S')
AND to_date = '%s'
'''
# AND to_price >= fm_price
sql = sql % today
print(sql)

df_stock = pd.read_sql(sql, con_lite)
df_stock.sort_values(['name'],ascending=[True]).style.format(format_dict)


SELECT T.name,sell_target AS target, sh, T.qty,T.available_qty,cost,
A.fm_date, A.to_date, days, fm_price, to_price, ttl_spread AS ttl, 
A.max_price AS max, T.max_price AS max52, consensus_price AS css_price,
sell_target * T.qty AS amount, T.status
FROM stocks T
JOIN sales A
ON T.name = A.name
LEFT JOIN consensus C
ON T.name = C.name
WHERE T.status IN ('I','S')
AND to_date = '2022-01-17'



,name,target,sh,qty,available_qty,cost,fm_date,to_date,days,fm_price,to_price,ttl,max,max52,css_price,amount,status
0,CKP,5.50,0,"10,000","30,000",5.00,2022-01-10,2022-01-17,5,4.98,5.10,3,5.10,6.30,6.45,"55,000.00",I
10,EPG,12.90,24H,"3,000","12,000",11.20,2022-01-14,2022-01-17,1,12.30,11.90,-4,12.30,13.70,14.79,"38,700.00",I
1,GLOBAL,21.00,0,"2,500","7,500",20.00,2022-01-13,2022-01-17,2,20.80,20.60,-2,20.80,24.70,27.31,"52,500.00",I
11,IVL,50.50,12,"3,000","12,000",42.00,2022-01-14,2022-01-17,1,48.50,47.75,-3,48.75,50.00,55.49,"151,500.00",I
12,KBANK,150.50,0,800,800,136.50,2022-01-14,2022-01-17,1,145.00,144.50,-1,145.00,153.00,175.20,"120,400.00",I
2,LPF,14.30,0,"10,000","10,000",14.00,2022-01-11,2022-01-17,4,13.90,14.00,1,14.20,0.00,14.90,"143,000.00",I
3,NER,7.80,0,"9,000","36,000",7.45,2022-01-13,2022-01-17,2,7.65,7.60,-1,7.70,8.65,11.35,"70,200.00",I
4,RCL,53.50,0,"1,500","6,000",44.50,2022-01-13,2022-01-17,2,53.25,48.00,-21,50.25,67.75,58.00,"80,250.00",I
5,RJH,36.25,0,"1,500","4,500",33.00,2022-01-10,2022-01-17,5,33.50,33.00,-2,33.25,39.00,36.50,"54,375.00",I
6,SAT,25.25,0,"2,500","5,000",21.10,2022-01-13,2022-01-17,2,24.50,23.40,-11,24.30,23.90,27.75,"63,125.00",I


In [5]:
my_array = df_stock.to_numpy()
my_array

array([['CKP', 5.5, 0, 10000, 30000, 5.0, '2022-01-10', '2022-01-17', 5,
        4.98, 5.1, 3, 5.1, 6.3, 6.45, 55000.0, 'I'],
       ['GLOBAL', 21.0, 0, 2500, 7500, 20.0, '2022-01-13', '2022-01-17',
        2, 20.8, 20.6, -2, 20.8, 24.7, 27.31, 52500.0, 'I'],
       ['LPF', 14.3, 0, 10000, 10000, 14.0, '2022-01-11', '2022-01-17',
        4, 13.9, 14.0, 1, 14.2, 0.0, 14.9, 143000.0, 'I'],
       ['NER', 7.8, 0, 9000, 36000, 7.45, '2022-01-13', '2022-01-17', 2,
        7.65, 7.6, -1, 7.7, 8.65, 11.35, 70200.0, 'I'],
       ['RCL', 53.5, 0, 1500, 6000, 44.5, '2022-01-13', '2022-01-17', 2,
        53.25, 48.0, -21, 50.25, 67.75, 58.0, 80250.0, 'I'],
       ['RJH', 36.25, 0, 1500, 4500, 33.0, '2022-01-10', '2022-01-17', 5,
        33.5, 33.0, -2, 33.25, 39.0, 36.5, 54375.0, 'I'],
       ['SAT', 25.25, 0, 2500, 5000, 21.1, '2022-01-13', '2022-01-17', 2,
        24.5, 23.4, -11, 24.3, 23.9, 27.75, 63125.0, 'I'],
       ['SYNEX', 37.0, 12, 1500, 4500, 31.5, '2022-01-13', '2022-01-17',
        

In [6]:
tmp = (np.apply_along_axis(number_of_spread, axis=1, arr=my_array))
tmp

array([ 8,  4,  2,  4, 22, 13, 17, 13, 11,  5, 10, 11, 12, 11])

In [7]:
df_stock['spd'] = tmp.tolist()
df_stock['spd'] = df_stock['spd'].astype(int)
df_stock.style.format(format_dict)

,name,target,sh,qty,available_qty,cost,fm_date,to_date,days,fm_price,to_price,ttl,max,max52,css_price,amount,status,spd
0,CKP,5.50,0,"10,000","30,000",5.00,2022-01-10,2022-01-17,5,4.98,5.10,3,5.10,6.30,6.45,"55,000.00",I,8
1,GLOBAL,21.00,0,"2,500","7,500",20.00,2022-01-13,2022-01-17,2,20.80,20.60,-2,20.80,24.70,27.31,"52,500.00",I,4
2,LPF,14.30,0,"10,000","10,000",14.00,2022-01-11,2022-01-17,4,13.90,14.00,1,14.20,0.00,14.90,"143,000.00",I,2
3,NER,7.80,0,"9,000","36,000",7.45,2022-01-13,2022-01-17,2,7.65,7.60,-1,7.70,8.65,11.35,"70,200.00",I,4
4,RCL,53.50,0,"1,500","6,000",44.50,2022-01-13,2022-01-17,2,53.25,48.00,-21,50.25,67.75,58.00,"80,250.00",I,22
5,RJH,36.25,0,"1,500","4,500",33.00,2022-01-10,2022-01-17,5,33.50,33.00,-2,33.25,39.00,36.50,"54,375.00",I,13
6,SAT,25.25,0,"2,500","5,000",21.10,2022-01-13,2022-01-17,2,24.50,23.40,-11,24.30,23.90,27.75,"63,125.00",I,17
7,SYNEX,37.00,12,"1,500","4,500",31.50,2022-01-13,2022-01-17,2,37.00,33.75,-13,34.75,36.50,28.67,"55,500.00",I,13
8,TOP,55.00,0,"3,000","3,000",50.00,2022-01-12,2022-01-17,3,52.75,52.25,-2,52.75,66.00,59.60,"165,000.00",I,11
9,WHART,12.80,0,"10,000","10,000",12.50,2022-01-11,2022-01-17,4,12.10,12.30,2,12.50,14.08,14.60,"128,000.00",I,5


In [8]:
sql = '''
SELECT name, volbuy AS shares, price AS unit_cost, volbuy * price AS cost_amt, dividend,
volbuy * dividend AS div_amt, (volbuy * dividend)/(volbuy * price) * 100 AS cost_pct, period
FROM buy
WHERE period IN ("1","2","3","4")
'''
df_buy = pd.read_sql(sql, const)
df_buy['shares'] = df_buy.shares.astype(int)
df_buy.sort_values(['name'],ascending=[True]).style.format(format_dict)

,name,shares,unit_cost,cost_amt,dividend,div_amt,cost_pct,period
10,BCH,"15,000",21.30,"319,500.00",0.33,"4,950.00",1.55,3
30,BGRIM,"9,000",40.00,"360,000.00",0.45,"4,050.00",1.12,4
19,CKP,"30,000",5.00,"150,000.00",0.04,"1,050.00",0.70,3
6,DIF,"60,000",14.70,"882,000.00",1.04,"62,640.00",7.10,2
12,DOHOME,"12,000",24.30,"291,600.00",0.10,"1,200.00",0.41,3
26,EPG,"12,000",11.20,"134,400.00",0.23,"2,760.00",2.05,3
8,GLOBAL,"7,500",20.00,"150,000.00",nan,nan,nan,3
17,HREIT,"30,000",8.95,"268,500.00",0.69,"20,796.00",7.75,2
28,IMH,"9,000",18.00,"162,000.00",nan,nan,nan,3
34,IP,"10,500",20.20,"212,100.00",0.18,"1,942.50",0.92,3


In [9]:
df_merge = pd.merge(df_stock, df_buy, on=['name'], how='inner')
cols = 'name spd target sh qty available_qty cost fm_price to_price ttl max max52 css_price status period amount'.split()
df_merge[cols].style.format(format_dict)

,name,spd,target,sh,qty,available_qty,cost,fm_price,to_price,ttl,max,max52,css_price,status,period,amount
0,CKP,8,5.50,0,"10,000","30,000",5.00,4.98,5.10,3,5.10,6.30,6.45,I,3,"55,000.00"
1,GLOBAL,4,21.00,0,"2,500","7,500",20.00,20.80,20.60,-2,20.80,24.70,27.31,I,3,"52,500.00"
2,LPF,2,14.30,0,"10,000","10,000",14.00,13.90,14.00,1,14.20,0.00,14.90,I,4,"143,000.00"
3,NER,4,7.80,0,"9,000","36,000",7.45,7.65,7.60,-1,7.70,8.65,11.35,I,4,"70,200.00"
4,RCL,22,53.50,0,"1,500","6,000",44.50,53.25,48.00,-21,50.25,67.75,58.00,I,3,"80,250.00"
5,RJH,13,36.25,0,"1,500","4,500",33.00,33.50,33.00,-2,33.25,39.00,36.50,I,4,"54,375.00"
6,SAT,17,25.25,0,"2,500","5,000",21.10,24.50,23.40,-11,24.30,23.90,27.75,I,4,"63,125.00"
7,SYNEX,13,37.00,12,"1,500","4,500",31.50,37.00,33.75,-13,34.75,36.50,28.67,I,3,"55,500.00"
8,TOP,11,55.00,0,"3,000","3,000",50.00,52.75,52.25,-2,52.75,66.00,59.60,I,3,"165,000.00"
9,WHART,5,12.80,0,"10,000","10,000",12.50,12.10,12.30,2,12.50,14.08,14.60,I,4,"128,000.00"


In [10]:
sell_alert = df_merge[cols]
sell_alert.set_index(['name'],inplace=True)
sell_alert.sort_values(['spd'],ascending=[True])

,spd,target,sh,qty,available_qty,cost,fm_price,to_price,ttl,max,max52,css_price,status,period,amount
name,,,,,,,,,,,,,,,
LPF,2,14.30,0,10000,10000,14.00,13.90,14.00,1,14.20,0.00,14.90,I,4,143000.0
GLOBAL,4,21.00,0,2500,7500,20.00,20.80,20.60,-2,20.80,24.70,27.31,I,3,52500.0
NER,4,7.80,0,9000,36000,7.45,7.65,7.60,-1,7.70,8.65,11.35,I,4,70200.0
WHART,5,12.80,0,10000,10000,12.50,12.10,12.30,2,12.50,14.08,14.60,I,4,128000.0
CKP,8,5.50,0,10000,30000,5.00,4.98,5.10,3,5.10,6.30,6.45,I,3,55000.0
EPG,10,12.90,24H,3000,12000,11.20,12.30,11.90,-4,12.30,13.70,14.79,I,3,38700.0
TOP,11,55.00,0,3000,3000,50.00,52.75,52.25,-2,52.75,66.00,59.60,I,3,165000.0
IVL,11,50.50,12,3000,12000,42.00,48.50,47.75,-3,48.75,50.00,55.49,I,3,151500.0
SIS,11,46.50,24,1500,4500,40.50,45.50,43.75,-7,45.75,48.75,36.88,I,3,69750.0


In [11]:
file_name = 'sell-alert.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

sell_alert.sort_values(['spd'],ascending=[True]).to_csv(output_file)
sell_alert.sort_values(['spd'],ascending=[True]).to_csv(data_file)
sell_alert.sort_values(['spd'],ascending=[True]).to_csv(box_file)